In [1]:
using ODE
using ForwardDiff # Mostly relies on package DualNumbers -> check in more detail later
using Formatting # For clear and concise printing output
using JLD
using NBInclude


# Subfunctions used for MBAM

In [9]:
# Return f(x) given log(x)
function log_deriv_wrapper(f::Function, phi_in::AbstractArray; log_specific=ones(size(phi_in)) )
    y = copy(phi_in)
    # Assuming some of the input parameters (x) are in log space, passes the original (exponentiated) values on to the function f
    if sum(log_specific==1)==prod(size(y)) # all the parameters are in log space
        y = exp(y)
    else
        for i1 = 1:size(log_specific)[1]
            if log_specific[i1]==1
                y[i1]=exp(y[i1])
            end
        end
    end
    
    return f(y)
end

log_deriv_wrapper (generic function with 1 method)

In [10]:
# Convert a parameter vector (or specific parameters) to log values
function to_log(phi_in::AbstractArray; log_specific=ones(size(phi_in)))
    y = copy(phi_in) # To avoid passing by reference!!!
    
    if sum(log_specific==1)==prod(size(y)) # all the parameters are in log space
        y = log(y)
    else
        for i1 = 1:size(log_specific)[1]
            if log_specific[i1]==1
                y[i1]=log(y[i1])
            end
        end
    end
    
    return y
end

to_log (generic function with 1 method)

In [11]:
# Given parameters x, selected parameters d and a canvas c, draw a scatter point at x[d] on the d-dim canvas c
#function scatter_param(x::Vector, d, c)
 #   Plots.plot(x[d])
#end

# Matlab-like rcond for matrix inversion
rcond(A::StridedMatrix) = LAPACK.gecon!('1', lufact(A).factors, norm(A, 1))

# Get a timestamp in useful format
timestamp()=Dates.format(now(),"_yyyymmddTHHMMSSss")

# Return an array as a string in a given format s for each element
function print_arr(s::AbstractString, x::AbstractArray)
    out = "[";
    for i1=1:length(x)
        out = string(out, sprintf1(s, x[i1]));
        if i1<length(x)
            out = string(out, ", ");
        end
    end
    out = string(out, "]");
    return out
end

print_arr (generic function with 1 method)

### Geodesic differential equation

As in Transtrum 2015 Supplemental

$$
\frac{d}{d\tau} v = - [\mathbf{J}^T \mathbf{J}]^{-1} \mathbf{J}^T \mathbf{A} \\
\mathbf{A}_m = v^T \mathbf{H_m} v \\
\frac{d}{d\tau} \phi_{cur} = v
$$

In [12]:
# Defines the geodesic differential equation with respect to a cost function f_cost and residual function f_res
function geodesic_ode(t, y, f_cost::Function, f_res::Function; verbose=1, move_dir=1.0, maxEV=1e-8, maxVnorm=[5e-2, 1e-3])
    sz2 = div(size(y)[1], 2);
    phi_cur = y[1:sz2]; # First half of vector is d-dim position
    v = y[(sz2+1):end]; # Second half of y vector is d-dim velocity
    
    #result = HessianResult(phi_cur);
    #ForwardDiff.hessian!(result, f_cost, phi_cur);
    
    #grad = ForwardDiff.gradient(result); # Gradient in parameter space
    #hess = ForwardDiff.hessian(result); # Hessian in parameter space
    jac = ForwardDiff.jacobian(f_res, phi_cur);
    f_jac(phi_cur,m) = map(Real, ForwardDiff.jacobian(f_res, phi_cur)[m,:])
    hess_m = cell(size(jac)[1],1); # Stores the second derivative of the m-th residual 
    A = zeros(size(jac)[1],1)
    #Consider doing this parallelly on multiple workers, one per data point
    for m1 = 1:(size(jac)[1])
        f_jac_m(phi_cur) = f_jac(phi_cur,m1);
        hess_m[m1] = ForwardDiff.jacobian(f_jac_m, phi_cur)
        A[m1] = (v'*hess_m[m1]*v)[1];
    end
    
    jac = map(Real, jac)
    
    (D,V) = eig(jac'*jac)
    
    # Printing intermediate steps
    if verbose>=2
        #@printf("size(A)=%s\n", size(A))
        #@printf("size(J)=%s\n", size(jac))
        @printf("smallest EV = %7.5e\n", sort(D)[1])
        println("   cost = $(f_cost(phi_cur))")
        println("phi_cur = $(print_arr("%10.2f", phi_cur))")
        println("      v = $(print_arr("%10.2e", v/norm(v)))")
        println()
        println()
    end
    
    #Stopping condition (small EV and the normalized v has "converged" - norm dominated by a few dimensions)
    vn = (v/norm(v));
    converged = (sort(D)[1] < maxEV)*(norm(vn[abs(vn).<maxVnorm[1]])<maxVnorm[2])
    if converged
        dy = copy(y);
        dy[1:end]=0
    else   
        dy = copy(y);
        dy[1:(sz2)] = move_dir * v # Change in phi_cur is v (or -v, change move_dir)
        dy[(sz2+1):end] = - move_dir * (jac'*jac) \ (jac' * A) # dy[2] is change in v (as y[2] is v), change in v is dy[1]: the second derivative
        #print(grad)
    end
    
    ## Matlab version (just take a step towards the smallest eigenvector)
    #dy[1:(sz/2)] = V[:,sortperm(D)[1]]
    #dy[(sz/2+1):end] = 0;
    return dy
end

geodesic_ode (generic function with 1 method)

# MBAM main function

In [1]:
# Runs automatic model reduction on a given model, starting at initial parameters phi0, evaluated at data point locations x_loc (Array of column vectors)
function MBAM(model::Function, phi0::AbstractArray, x_loc::AbstractArray; model_iters=0, boundary_time=10, log_specific=ones(size(phi0)), verbose=1, 
    move_dir=ones(size(model_iters)), maxEV=1e-8, maxVnorm=[5e-2, 1e-3], reduce_func=[], prevRunResults=[], data_vec=[])
    #maxEV - stops ode solver if smallest EV of J'J < maxEV AND
    #maxVnorm - stops ode solver if norm(vn[abs(vn).<maxVnorm[1]])<maxVnorm[2]
    
    
    phi_red = to_log(phi0, log_specific=log_specific);
    x_loc = copy(x_loc)
    x_loc = hcat(x_loc...)
    #@show x_loc[1]
    # Initialize variables we want to remember in the end (otherwise their scope is limited to within the for cycle)
    phi_cur = phi_red;
    V = phi_red;
    D = phi_red;
    v = phi_red;
    
    # Variables to keep
    phis = Array(Any, (length(model_iters),2)) # Save param values before/after geodesic
    Ds = Array(Any, (length(model_iters),2)) # Save eigendecomps before/after geodesic
    Vs = Array(Any, (length(model_iters),2)) # Save eigendecomps before/after geodesic
    costs = Array(Any, (length(model_iters),2)) # Save eigendecomps before/after geodesic
    t_out = Array(Any, length(model_iters))
    y_out = Array(Any, length(model_iters))
    model_reduced = Array(Any, length(model_iters))
    cur_time = timestamp();
    
    model_cur(phi::AbstractArray, x::AbstractArray) = model(phi, x, model_id=model_iters[1])
    model_orig = copy(model_cur);
    
    if isempty(data_vec)
        @time data_vec = model_orig(phi0, x_loc)[:]; # Compute the original data only once
    end
    
    for n1 = 1:length(model_iters) # Outer loop, reduces the number of parameters in the model each step
        m1 = model_iters[n1]
        # Define current cost function and residual function
        f_cost = phi1 -> sum((model_cur(phi1, x_loc)[:].-data_vec).^2); # Current cost function (quadratic)
        g_cost = phi1 -> log_deriv_wrapper(f_cost, phi1, log_specific=log_specific); # Define which parameters we're taking in log space ([default: all])
        f_res = phi1 -> (model_cur(phi1, x_loc)[:].-data_vec) # Return residuals
        g_res = phi1 -> log_deriv_wrapper(f_res, phi1, log_specific=log_specific);
        
        phi_cur = phi_red; # Initial point in parameter space
        
        #@show size(model_cur(phi_cur, x_loc)[:])
        #@show size(model_orig(phi0, x_loc)[:])
        
        #@show g_res(phi_cur)
        println("Model iteration $(m1), Single g_cost call takes: ")
        @time g_cost(phi_cur)
        
        jac = map(Real, ForwardDiff.jacobian(g_res, phi_cur));
        #@show model_cur(phi_cur, x_loc)
        
        (D,V) = eig(jac'*jac)
        
        # Store for output (before geodesic)
        phis[n1,1] = phi_cur;
        costs[n1,1] = g_cost(phi_cur)
        Ds[n1,1] = D;
        Vs[n1,1] = V;
        
        
        v = V[:,sortperm(D)[1]]; # Initial velocity in parameter space
        
        if verbose>=1
            println("Model iteration $(m1), initial values:")
            println("      D = $(print_arr("%10.2e", D))")
            println("   cost = $(g_cost(phi_cur))")
            println("phi_cur = $(print_arr("%10.2f", phi_cur))")
            println("      v = $(print_arr("%10.2e", v))")
            
            println()
            println()
        end
        
    
        if !isempty(prevRunResults) && (n1<=(size(prevRunResults[1])[1]))
            # Grab the data from the results array of a previous run
            phi_cur = prevRunResults[1][n1,2];
            v = prevRunResults[4][n1][:,sortperm(prevRunResults[3][n1,2])[1]];
            v = v/norm(v) * move_dir[n1];
            t_out[n1] = prevRunResults[5][n1]
            y_out[n1] = prevRunResults[6][n1]
            
            phis[n1,2] = phi_cur;
            costs[n1,2] = g_cost(phi_cur)
            Ds[n1,2] = prevRunResults[3][n1,2];
            Vs[n1,2] = prevRunResults[4][n1,2];
        else
        
            # Run the ODE solver
            t_out[n1], y_out[n1] = ODE.ode23((t,y) -> geodesic_ode(t,y,g_cost,g_res,verbose=verbose, move_dir=move_dir[find(model_iters.==m1)[1]], maxEV=maxEV, maxVnorm=maxVnorm), [phi_cur; v], [0,boundary_time])

            phi_cur = y_out[n1][end][1:size(phi_cur)[1]] # Final point in parameter space (on the boundary)
            v = y_out[n1][end][(size(phi_cur)[1]+1):end] # Final velocity in parameter space (reduceable combination)
            v = v*move_dir[n1]
            v = v/norm(v)


            jac = ForwardDiff.jacobian(g_res, phi_cur);
            (D,V) = eig(jac'*jac)

            # Store for output (after geodesic)
            phis[n1,2] = phi_cur;
            costs[n1,2] = g_cost(phi_cur)
            Ds[n1,2] = D;
            Vs[n1,2] = V;
        end       
        
        if verbose>=1
            println("Model iteration $(m1), final values:")
            println("      D = $(print_arr("%10.2e", D))")
            println("   cost = $(g_cost(phi_cur))")
            println("phi_cur = $(print_arr("%10.2f", phi_cur))")
            println("      v = $(print_arr("%10.2e", v))")
            println()
            println()
        end
    
        #@show size(x_loc[1])
        
        
        
        if typeof(reduce_func)!=Function
            # Do the model reduction symbolicly
            if m1==model_iters[1]
                @time model_reduced[n1] = Reduce_model(model, exp(phi_cur), v, (size(x_loc)[1],), model_id=m1)
            else
                @time model_reduced[n1] = Reduce_model(model_reduced[n1-1][2], exp(phi_cur), v, (size(x_loc)[1],))
            end
            
            @show model_reduced[n1][2] # Show the reduced model (symbolic equation)
            @show model_reduced[n1][3] # Show the reduction mapping

            # Sort out the results
            model_cur = model_reduced[n1][4]
            log_specific = log_specific[1:length(model_reduced[n1][end])]
            phi_red = to_log(model_reduced[n1][end], log_specific=log_specific)
        else
            
            # Use the provided reduction function and model_id
            
            phi_red = reduce_func(phi_cur, model_id=model_iters[n1]);
            if n1<length(model_iters)
                model_cur(phi::AbstractArray, x::AbstractArray) = model(phi, x, model_id=model_iters[n1+1])
                log_specific = log_specific[1:length(phi_red)]
            end
        end
    end

    
    return phis, costs, Ds, Vs, t_out, y_out, model_reduced
end #line 35

LoadError: LoadError: UndefVarError: @save not defined
while loading In[1], in expression starting on line 96